# <center>Đồ án 3: Linear Regression</center>


# Thông tin sinh viên

-   Họ và tên: Nguyễn Văn Đạt
-   MSSV: 20127132
-   Lớp: 20CLC5


# Import


In [ ]:
import pandas as pd
import numpy as np

# Import thêm dữ thư viện nếu cần
from sklearn.utils import shuffle
import matplotlib.pyplot as plt


# Đọc dữ liệu


In [ ]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
# Dataframe (chứa 10 đặc trưng huấn luyện)
X_train = train.iloc[:, :-1].to_numpy()
# Series    (chứa 1 giá trị mục tiêu kiểm tra)
y_train = train.iloc[:, -1].to_numpy()

# Dataframe (chứa 10 đặc trưng kiểm tra)
X_test = test.iloc[:, :-1].to_numpy()
# Series    (chứa 1 giá trị mục tiêu kiểm tra)
y_test = test.iloc[:, -1].to_numpy()

# Sinh viên có thể sử dụng các khác nếu cần
title = ["Adult Mortality", 'BMI', 'Polio', 'Diphtheria', "HIV/AIDS", 'GDP',
         'Thinness age 10-19', 'Thinness age 5-9', 'Income composition of resources', 'Schooling']


# Cài đặt hàm


In [ ]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y
        return self

    def get_params(self):
        return self.w

    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)


def plot_regression(lr, x, y):
    index = 0
    # Plot data
    plt.scatter(x[:, index], y, color='b', marker='o')
    index += 1
    if(index < x.shape[1]):
        plt.scatter(x[:, index], y, color='b', marker='o')
        index += 1
    if(index < x.shape[1]):
        plt.scatter(x[:, index], y, color='b', marker='o')

    # Plot regression line
    x_para = np.linspace(0, 100, 1085).reshape(-1, 1)
    if x.shape[1] == 1:
        X_para = np.hstack((x_para**2))
    if x.shape[1] == 3:
        X_para = np.hstack((x_para, x_para**2, x_para**3))
    else:
        X_para = np.hstack((x_para, x_para**2))
    y_pred = lr.predict(X_para)
    plt.plot(x_para, y_pred, color='r')

    plt.xlabel('x')
    plt.ylabel('y')

    plt.grid()
    plt.show()


def RMSE(y_test, y_test_pred):
    return np.sqrt(np.square(np.subtract(y_test, y_test_pred)).mean())


# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm)


In [ ]:
# Phần code cho yêu cầu 1a
def main1a(X_train, y_train, X_test):
    x = X_train
    y = y_train
    lr = OLSLinearRegression().fit(x, y)
    y_test_pred = lr.predict(X_test)
    return y_test_pred


In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
rmse = RMSE(y_test, main1a(X_train, y_train, X_test))
print("RMSE =", rmse)


Công thức hồi quy

$$
\text{Life expectancy} = 0.0151 * Adult Mortality + 0.0902 * BMI + 0.0429 * Polio + 0.1393 * Diphtheria - 0.5673 * HIV/AIDS - 0.0001 * GDP - 0.7407 * (Thinness age 10-19) - 0.1909 * (Thinness age 5-9) + 24.5060 * (Income composition of resources) + 2.39350 * Schooling
$$


# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)


Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng


In [ ]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
def main1b(X_train, y_train):
    X_train_clone = X_train.copy()
    y_train_clone = y_train.copy()

    X_train_clone, y_train_clone = shuffle(
        X_train_clone, y_train_clone, random_state=0)

    rmse_mean = []
    for i in range(0, 10):

        X_train_feature = X_train_clone[:, i].copy().reshape(-1, 1)

        y_train_feature = y_train_clone.copy().reshape(-1, 1)

        rmse = []
        for k in range(0, 5):
            if k == 0:
                x_k_val = X_train_feature[:217]
                y_k_val = y_train_feature[:217]
                x_k_train = X_train_feature[217:]
                y_k_train = y_train_feature[217:]
            elif k == 4:
                x_k_val = X_train_feature[(k*217):]
                y_k_val = y_train_feature[(k*217):]
                x_k_train = X_train_feature[:(k*217)]
                y_k_train = y_train_feature[:(k*217)]
            else:
                x_k_val = X_train_feature[(k*217):(k*217) + 217]
                y_k_val = y_train_feature[(k*217):(k*217) + 217]

                x_k_train = np.concatenate(
                    (X_train_feature[((k + 1)*217):], X_train_feature[:(k*217)]), axis=None).reshape(-1, 1)
                y_k_train = np.concatenate(
                    (y_train_feature[((k + 1)*217):], y_train_feature[:(k*217)]), axis=None).reshape(-1, 1)

            lr = OLSLinearRegression().fit(x_k_train, y_k_train)
            y_k_val_pred = lr.predict(x_k_val)

            rmse.append(RMSE(y_k_val, y_k_val_pred))

        rmse_mean.append(np.mean(rmse))
    return rmse_mean


In [ ]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

def re_train_best_feature_model(X_train, y_train, X_test):

    i_best = np.argmin(main1b(X_train, y_train), axis=0)

    print('Sử dụng đặc trưng:', title[i_best])

    X_re_train = X_train[:, i_best].copy().reshape(-1, 1)

    y_re_train = y_train.copy().reshape(-1, 1)

    X_test_re_train = X_test[:, i_best].copy().reshape(-1, 1)

    x_best_feature = X_re_train

    lr = OLSLinearRegression().fit(x_best_feature, y_re_train)

    y_test_pre = lr.predict(X_test_re_train)

    return y_test_pre


In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
rmse = RMSE(y_test, re_train_best_feature_model(X_train, y_train, X_test))
print("RMSE =",rmse)


<center>

| STT |     Mô hình với 1 đặc trưng     |  RMSE   |
| :-: | :-----------------------------: | :-----: |
|  1  |         Adult Mortality         | 41.9200 |
|  2  |               BMI               | 33.2783 |
|  3  |              Polio              | 21.1875 |
|  4  |           Diphtheria            | 19.5753 |
|  5  |            HIV/AIDS             | 65.6792 |
|  6  |               GDP               | 62.4901 |
|  7  |       Thinness age 10-19        | 48.9019 |
|  8  |        Thinness age 5-9         | 48.8179 |
|  9  | Income composition of resources | 20.9910 |
| 10  |            Schooling            | 19.1516 |

<center>


Công thức hồi quy

$$\text{Life expectancy} = 5.5573 * Schooling$$


# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)


Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế


In [ ]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

def model1(X_train, i, j):
    return np.concatenate((X_train[:, i].copy(
    ).reshape(-1, 1), X_train[:, j].copy().reshape(-1, 1))).reshape(-1, 2)


def model2(X_train, i, j, k):
    return np.concatenate(((X_train[:, i].copy(
    ).reshape(-1, 1)), X_train[:, j].copy().reshape(-1, 1), X_train[:, k].copy().reshape(-1, 1))).reshape(-1, 3)


def model3(X_train, i, j, k):
    return np.concatenate((X_train[:, i].copy().reshape(-1, 1) * X_train[:, j].copy(
    ).reshape(-1, 1), X_train[:, k].copy().reshape(-1, 1))).reshape(-1, 2)


def main1c_1(X_train, y_train):
    X_train_clone = X_train.copy()
    y_train_clone = y_train.copy()

    X_train_clone, y_train_clone = shuffle(
        X_train_clone, y_train_clone, random_state=0)

    rmse_mean = []
    rmse_mean_pos = []

    for i in range(0, 9):
        for j in range(i + 1, 10):

            X_train_feature = model1(X_train_clone, i, j)

            y_train_feature = y_train_clone.copy().reshape(-1, 1)

            rmse = []

            for k in range(0, 5):
                if k == 0:
                    x_k_val = X_train_feature[:217, :]
                    y_k_val = y_train_feature[:217]
                    x_k_train = X_train_feature[217:, :]
                    y_k_train = y_train_feature[217:]
                elif k == 4:
                    x_k_val = X_train_feature[(k*217):]
                    y_k_val = y_train_feature[(k*217):]
                    x_k_train = X_train_feature[:(k*217)]
                    y_k_train = y_train_feature[:(k*217)]
                else:
                    x_k_val = X_train_feature[(k*217):(k*217) + 217]
                    y_k_val = y_train_feature[(k*217):(k*217) + 217]

                    x_k_train = np.concatenate(
                        (X_train_feature[((k + 1)*217):], X_train_feature[:(k*217)]), axis=0).reshape(-1, 2)
                    y_k_train = np.concatenate(
                        (y_train_feature[((k + 1)*217):], y_train_feature[:(k*217)]), axis=0).reshape(-1, 1)

                lr = OLSLinearRegression().fit(x_k_train, y_k_train)
                y_k_val_pred = lr.predict(x_k_val)

                rmse.append(RMSE(y_k_val, y_k_val_pred))

            rmse_mean.append(np.mean(rmse))
            rmse_mean_pos.append([i, j])

    return rmse_mean, rmse_mean_pos


def main1c_2(X_train, y_train):
    X_train_clone = X_train.copy()
    y_train_clone = y_train.copy()

    X_train_clone, y_train_clone = shuffle(
        X_train_clone, y_train_clone, random_state=0)

    rmse_mean = []
    rmse_mean_pos = []

    for i in range(0, 8):
        for j in range(i+1, 9):
            for l in range(j+1, 10):
                X_train_feature = model2(X_train_clone, i, j, l)

                y_train_feature = y_train_clone.copy().reshape(-1, 1)

                rmse = []

                lr = OLSLinearRegression()

                for k in range(0, 5):
                    if k == 0:
                        x_k_val = X_train_feature[:217, :]
                        y_k_val = y_train_feature[:217]
                        x_k_train = X_train_feature[217:, :]
                        y_k_train = y_train_feature[217:]
                    elif k == 4:
                        x_k_val = X_train_feature[(k*217):]
                        y_k_val = y_train_feature[(k*217):]
                        x_k_train = X_train_feature[:(k*217)]
                        y_k_train = y_train_feature[:(k*217)]
                    else:
                        x_k_val = X_train_feature[(k*217):(k*217) + 217]
                        y_k_val = y_train_feature[(k*217):(k*217) + 217]

                        x_k_train = np.concatenate(
                            (X_train_feature[((k + 1)*217):], X_train_feature[:(k*217)]), axis=0).reshape(-1, 3)
                        y_k_train = np.concatenate(
                            (y_train_feature[((k + 1)*217):], y_train_feature[:(k*217)]), axis=0).reshape(-1, 1)

                    lr = lr.fit(x_k_train, y_k_train)

                    y_k_val_pred = lr.predict(x_k_val)

                    rmse.append(RMSE(y_k_val, y_k_val_pred))

                rmse_mean.append(np.mean(rmse))
                rmse_mean_pos.append([i, j, l])

    return rmse_mean, rmse_mean_pos


def main1c_3(X_train, y_train):
    X_train_clone = X_train.copy()
    y_train_clone = y_train.copy()

    X_train_clone, y_train_clone = shuffle(
        X_train_clone, y_train_clone, random_state=0)

    rmse_mean = []
    rmse_mean_pos = []

    for i in range(0, 10):
        for j in range(0, 10):
            if i == j:
                continue
            for l in range(0, 10):
                if i == l or j == l:
                    continue
                X_train_feature = model3(X_train_clone, i, j, l)

                y_train_feature = y_train_clone.copy().reshape(-1, 1)

                rmse = []

                for k in range(0, 5):
                    if k == 0:
                        x_k_val = X_train_feature[:217, :]
                        y_k_val = y_train_feature[:217]
                        x_k_train = X_train_feature[217:, :]
                        y_k_train = y_train_feature[217:]
                    elif k == 4:
                        x_k_val = X_train_feature[(k*217):]
                        y_k_val = y_train_feature[(k*217):]
                        x_k_train = X_train_feature[:(k*217)]
                        y_k_train = y_train_feature[:(k*217)]
                    else:
                        x_k_val = X_train_feature[(k*217):(k*217) + 217]
                        y_k_val = y_train_feature[(k*217):(k*217) + 217]

                        x_k_train = np.concatenate(
                            (X_train_feature[((k + 1)*217):], X_train_feature[:(k*217)]), axis=0).reshape(-1, 2)
                        y_k_train = np.concatenate(
                            (y_train_feature[((k + 1)*217):], y_train_feature[:(k*217)]), axis=0).reshape(-1, 1)

                    lr = OLSLinearRegression().fit(x_k_train, y_k_train)
                    y_k_val_pred = lr.predict(x_k_val)

                    rmse.append(RMSE(y_k_val, y_k_val_pred))

                rmse_mean.append(np.mean(rmse))
                rmse_mean_pos.append([i, j, l])

    return rmse_mean, rmse_mean_pos


<center>

| STT |                                   Mô hình                                 |  RMSE   |
| :-: | :-----------------------------------------------------------------------: | :-----: |
|  1  |                   Sử dụng 2 đặc trưng (Polio, Diphtheria)                 | 16.0253 |
|  2  |      Sử dụng 3 đặc trưng (BMI, Polio, Diphtheria)                         | 24.8921 |
|  3  | Sử dụng 3 đặc trưng (Diphtheria, Income composition of resources , Polio) | 23.9041 |

</center>


In [ ]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

def re_train_best_feature_model_1c_1(X_train, y_train, X_test, rmse_mean, rmse_mean_pos):

    i_best = rmse_mean_pos[np.argmin(rmse_mean)]

    print('Sử dụng 2 đặc trưng:', title[i_best[0]], ',', title[i_best[1]])

    X_re_train = model1(X_train, i_best[0], i_best[1])

    y_re_train = y_train.copy().reshape(-1, 1)

    X_test_re_train = model1(X_test, i_best[0], i_best[1])

    x_best_feature = X_re_train

    lr = OLSLinearRegression().fit(x_best_feature, y_re_train)

    y_test_pre = lr.predict(X_test_re_train)

    return y_test_pre


def re_train_best_feature_model_1c_2(X_train, y_train, X_test, rmse_mean, rmse_mean_pos):

    i_best = rmse_mean_pos[np.argmin(rmse_mean)]

    print('Sử dụng 3 đặc trưng:', title[i_best[0]],
          ',', title[i_best[1]], ',', title[i_best[2]])

    X_re_train = model2(X_train, i_best[0], i_best[1], i_best[2])

    y_re_train = y_train.copy().reshape(-1, 1)

    X_test_re_train = model2(X_test, i_best[0], i_best[1], i_best[2])

    x_best_feature = X_re_train

    lr = OLSLinearRegression().fit(x_best_feature, y_re_train)

    y_test_pre = lr.predict(X_test_re_train)

    return y_test_pre


def re_train_best_feature_model_1c_3(X_train, y_train, X_test, rmse_mean, rmse_mean_pos):

    i_best = rmse_mean_pos[np.argmin(rmse_mean)]

    print('Sử dụng 3 đặc trưng:', title[i_best[0]],
          ',', title[i_best[1]], ',', title[i_best[2]])

    X_re_train = model3(X_train, i_best[0], i_best[1], i_best[2])

    y_re_train = y_train.copy().reshape(-1, 1)

    X_test_re_train = model3(X_test, i_best[0], i_best[1], i_best[2])

    x_best_feature = X_re_train

    lr = OLSLinearRegression().fit(x_best_feature, y_re_train)

    y_test_pre = lr.predict(X_test_re_train)

    return y_test_pre


def re_train_best_feature_model_1c(X_train, y_train, X_test):

    rmse_mean1, rmse_mean_pos1 = main1c_1(X_train, y_train)
    rmse_mean2, rmse_mean_pos2 = main1c_2(X_train, y_train)
    rmse_mean3, rmse_mean_pos3 = main1c_3(X_train, y_train)
    rmse_best = np.argmin(
        [np.min(rmse_mean1), np.min(rmse_mean2), np.min(rmse_mean3)])

    if rmse_best == 0:
        print('Mô hình 1:')
        return re_train_best_feature_model_1c_1(X_train, y_train, X_test, rmse_mean1, rmse_mean_pos1)
    elif rmse_best == 1:
        print('Mô hình 2:')
        return re_train_best_feature_model_1c_2(X_train, y_train, X_test, rmse_mean2, rmse_mean_pos2)
    else:
        print('Mô hình 3:')
        return re_train_best_feature_model_1c_3(X_train, y_train, X_test, rmse_mean3, rmse_mean_pos3)


In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

rmse = RMSE(y_test, re_train_best_feature_model_1c(X_train, y_train, X_test))
print("RMSE =", rmse)


Công thức hồi quy

$$\text{Life expectancy} = 0.4398 * Polio + 0.3505 * Diphtheria$$